In [7]:
import pandas as pd
import pyodbc
import pyarrow as pa
import pyarrow.parquet as pq
import os
import subprocess
import shutil
import glob
from datetime import date, datetime
from time import time
from subprocess import Popen, PIPE

In [8]:
# Variáveis de data e hora
ref_data = date.today()
ref_year = date.today().year
ref_time = datetime.now().strftime("%H%M%S")

In [169]:
# Parametros para a criação da string de conexão ao banco sql
server = 'localhost'
database = 'db_datasus'
username = 'sa'
password = 'SenhaDev1234'

# String de conexão ao banco sql
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+password)

query = """SELECT [date] 
                ,[state] 
                ,SUM([confirmed]) AS [confirmed] 
                ,SUM([deaths]) AS [deaths] 
                ,[estimated_population_2019] 
                ,[estimated_population] 
                ,[city_ibge_code] 
                ,[confirmed_per_100k_inhabitants] 
                ,[death_rate] 
            FROM [db_datasus].[dbo].[COVID] 
            WHERE city_ibge_code < 100 
            GROUP BY  
                  [date] 
                  ,[state] 
                  ,[estimated_population_2019] 
                  ,[estimated_population] 
                  ,[city_ibge_code] 
                  ,[confirmed_per_100k_inhabitants] 
                  ,[death_rate] """

sql_query = pd.read_sql_query(query, conn)

In [170]:
# Cria o dataset pandas a partir da consulta sql
df = pd.DataFrame(sql_query)

In [171]:
# df.to_parquet('df.parquet.snappy',compression='snappy')
# pd.read_parquet('df.parquet.snappy')

In [172]:
# Realiza a conversa do dataset em pandas para uma table
table = pa.Table.from_pandas(df)

In [10]:
# Declara as variáveis de dia e hora corrente para montar o nome do arquivo
date = datetime.now().strftime('%Y%m%d')
time = str(ref_time)

# Pasta e nome do arquivo no diretório temporário
filename = '/app/hadoop/tmp/dfs/tmp/etl_hdfs'+'_'+date +'_'+ time

# # Convert a table para o arquivo .parquet
pq.write_table(table, filename+'.parquet')

'/app/hadoop/tmp/dfs/tmp/etl_hdfs_20210429_153019'

In [174]:
# Criação da função para realizando do PUT no HDFS
def run_cmd(args_list):
    """
    run linux commands
    """
    # import subprocess
    print('Running system command: {0}'.format(' '.join(args_list)))
    proc = subprocess.Popen(args_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    s_output, s_err = proc.communicate()
    s_return = proc.returncode
    return s_return, s_output, s_err

In [175]:
# Variáveis para criação do Caminho de Destino no HDFS
dir_ano = str(ref_year)
dir_mes = datetime.now().strftime('%m')
dir_dia = datetime.now().strftime('%d')

# Caminho de origem do arquivo .parquet
source_dir = '/app/hadoop/tmp/dfs/tmp/'
sources = glob.glob(os.path.join(source_dir,"*.parquet"))

# Caminho de destino do arquivo .parquet
(ret, out, err) = run_cmd(['hadoop', 'fs', '-put', sources[0], '/user/datalake/raw/'+str(dir_ano)+'/'+str(dir_mes)+'/'+str(dir_dia)])

Running system command: hadoop fs -put /app/hadoop/tmp/dfs/tmp/etl_hdfs_2021-04-29_150832.parquet /user/datalake/raw/2021/04/29


In [176]:
# Rotina para limpar a pasta Temp após o input no HDFS
source_dir = '/app/hadoop/tmp/dfs/tmp/'
sources = glob.glob(os.path.join(source_dir,"*.parquet"))

for f in sources:
    os.remove(f)